In [31]:
import psycopg2 as ps
from env_vars import *

In [41]:
def reconnect():
    # create connection and cursor    
    conn = ps.connect(host=POSTGRES_ADDRESS,
                      database=POSTGRES_DBNAME,
                      user=POSTGRES_USERNAME,
                      password=POSTGRES_PASSWORD,
                      port=POSTGRES_PORT)
    cur = conn.cursor()
    return cur

In [42]:
# def run_query():
#     try:
#         cur.execute(query)
#     except Exception as e:
#         print (e.message)
#         return reconnect()

In [43]:
# run_query()

In [ ]:
# query = """SELECT * FROM users
#             WHERE schemaname != 'pg_catalog'
#             AND schemaname != 'information_schema';"""
# cur.execute(query)
# cur.fetchall()

In [45]:
cur = reconnect()
query = """SELECT * FROM users;"""

cur.execute(query)
cur.fetchall()

[]

In [ ]:
# cur.execute("""CREATE TABLE users
#                 (ID integer, 
#                 display_name varchar(50)
#                 );""")
# # Commit table creation
# conn.commit()

In [51]:
cur = reconnect()
cur.execute("""CREATE TABLE recommendations
                (userID integer, 
                songID integer,
                recDate DATE 
                );""")
# Commit table creation
conn.commit()